In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

PRESET = os.environ.setdefault("PRESET", "test_discretizer")
SELF_PARTY = os.environ.setdefault("SELF_PARTY", "sim")
os.environ["NODE_ENV"] = "development"

In [ ]:
import ray
from opentelemetry import trace
from opentelemetry.sdk.environment_variables import OTEL_SERVICE_NAME

from secretnote.instrumentation.sdk import (
    create_profiler,
    inherit_tracing_context,
    setup_debug_exporter,
    setup_tracing,
)

ray.shutdown()

os.environ[OTEL_SERVICE_NAME] = f"{PRESET}:{SELF_PARTY}"

setup_tracing()
setup_debug_exporter()


with trace.get_tracer(__name__).start_as_current_span(
    "sim_trace",
    context=inherit_tracing_context(),
):
    with create_profiler() as profiler:
        with open(f"presets/{PRESET}/_world.py") as f:
            _world = compile(f.read(), f.name, "exec")
        with open(f"presets/{PRESET}/_algorithm.py") as f:
            _algorithm = compile(f.read(), f.name, "exec")
        exec(_world, globals())
        exec(_algorithm, globals())

In [4]:
from secretnote.display.digest import TimelineDigest

timeline = TimelineDigest()

for span in profiler.exporter.iter_spans():
    timeline.feed(span)

timeline.digest()

In [5]:
for frame in timeline.frames.values():
    if (
        frame.expression
        and len(frame.semantics) == 1
        and frame.semantics[0].api_level == 20
    ):
        print(frame.expression)

P(1) := PYU[alice]. let () in exec(<lambda>) | (df) + (index)
P(2) := PYU[bob]. let () in exec(<lambda>) | (df) + (index)
P(3) := PYU[alice]. let P(1) in exec(_fillna) | (value, method, axis, inplace, limit, downcast) + (inplace)
P(4) := PYU[bob]. let P(2) in exec(_fillna) | (value, method, axis, inplace, limit, downcast) + (inplace)
hdf_alice <== reveal P(3)
hdf_bob <== reveal P(4)
P(5) := PYU[alice]. let () in exec(<lambda>) | () + (vdf_alice)
P(6) := PYU[bob]. let () in exec(<lambda>) | () + (vdf_bob)
P(7) := PYU[alice]. let () in exec(<lambda>) | () + (df_part0)
P(8) := PYU[bob]. let () in exec(<lambda>) | () + (df_part1)
P(9) := PYU[alice]. let () in exec(<lambda>) | () + (df_part0)
P(10) := PYU[bob]. let () in exec(<lambda>) | () + (df_part1)
P(11) := PYU[alice]. let P(9) in exec(<lambda>) | () + ()
cols <== reveal P(11)
P(12) := PYU[bob]. let P(10) in exec(<lambda>) | () + ()
python/id/0x33bfd07c0+1 <== reveal P(12)
P(13) := PYU[alice]. let P(7) in exec(<lambda>) | () + ()
cols 